# Uniform Cost Search

In this exercise you'll implement extend breadth-first search by incorporating a cost for each action.

In [114]:
from queue import Queue, PriorityQueue
import numpy as np
from enum import Enum
import math

https://wiki.python.org/moin/TimeComplexity gives a solid overview of Python data structures and their time complexity. 

Is there a data structure more suitable for this task than a `Queue`?

* [`Enum`](https://docs.python.org/3/library/enum.html#module-enum) is used to represent possible actions on the grid.

In [115]:
class Action(Enum):
    """
    An action is represented by a 3 element tuple.
    
    The first 2 values are the delta of the action relative
    to the current grid position. The third and final value
    is the cost of performing the action.
    """
    LEFT = (0, -1, 1)
    RIGHT = (0, 1, 1)
    UP = (-1, 0, 1)
    DOWN = (1, 0, 1)
    UP_LEFT = (-1, -1, math.sqrt(2))
    UP_RIGHT = (-1, 1, math.sqrt(2))
    DOWN_LEFT = (1, -1, math.sqrt(2))
    DOWN_RIGHT = (1, 1, math.sqrt(2))
    
    def __str__(self):
        if self == self.LEFT:
            return '<'
        elif self == self.RIGHT:
            return '>'
        elif self == self.UP:
            return '^'
        elif self == self.DOWN:
            return 'v'
        elif self == self.UP_LEFT:
            return '<'
        elif self == self.UP_RIGHT:
            return '>'
        elif self == self.DOWN_LEFT:
            return '<'
        elif self == self.DOWN_RIGHT:
            return '>'
    
    @property
    def cost(self):
        return self.value[2]
    
    @property
    def delta(self):
        return (self.value[0], self.value[1])
            
    
def valid_actions(grid, current_node):
    """
    Returns a list of valid actions given a grid and current node.
    """
    valid = [Action.UP, Action.LEFT, Action.RIGHT, Action.DOWN, Action.UP_LEFT, Action.UP_RIGHT, Action.DOWN_LEFT, Action.DOWN_RIGHT]
    m, n = grid.shape[0] - 1, grid.shape[1] - 1
    y, x = current_node
    
    # check if the node is off the grid or
    # it's an obstacle
    
    if x - 1 < 0 or grid[y, x-1] == 1:
        valid.remove(Action.LEFT)
    if x + 1 > n or grid[y, x+1] == 1:
        valid.remove(Action.RIGHT)
    if y - 1 < 0 or grid[y-1, x] == 1:
        valid.remove(Action.UP)
    if y + 1 > m or grid[y+1, x] == 1:
        valid.remove(Action.DOWN)
    if (x - 1 < 0 or y - 1 < 0) or grid[y-1, x-1] == 1:
        valid.remove(Action.UP_LEFT)
    if (x + 1 > n or y - 1 < 0) or grid[y-1, x+1] == 1:
        valid.remove(Action.UP_RIGHT)
    if (x - 1 < 0 or y + 1 > m) or grid[y+1, x-1] == 1:
        valid.remove(Action.DOWN_LEFT)
    if (x + 1 > n or y + 1 > m) or grid[y+1, x+1] == 1:
        valid.remove(Action.DOWN_RIGHT)
    print(valid)
        
        
    return valid

def visualize_path(grid, path, start):
    sgrid = np.zeros(np.shape(grid), dtype=np.str)
    sgrid[:] = ' '
    sgrid[grid[:] == 1] = 'O'
    
    pos = start
    
    for a in path:
        da = a.value
        sgrid[pos[0], pos[1]] = str(a)
        pos = (pos[0] + da[0], pos[1] + da[1])
    sgrid[pos[0], pos[1]] = 'G'
    sgrid[start[0], start[1]] = 'S'  
    return sgrid

### Cost Search

In this section you will extend the breadth-first search algorithm by incorporating a cost for each action. Your task is to compute the lowest cost path. Does this change the data structures you should use?

You will need to implement the remaining `TODOs`.

In [116]:
def uniform_cost(grid, start, goal):

    # TODO: Initialize the starting variables
    path = []
    queue = PriorityQueue()
    visited = set()
    branch = {}
    found = False
    
    # Priority queue stores the position and the cost it took to get to that position
    queue.put((0, start))
    visited.add(start)
    
    while not queue.empty():
        current_node = queue.get()
        print(current_node)
        current_cost = current_node[0]
        current_pos = current_node[1]

        if current_pos == goal:        
            print('Found a path.')
            found = True
            break
        else:
            for action in valid_actions(grid, current_pos):
                delta_pos = action.delta
                new_pos = (current_pos[0] + delta_pos[0], current_pos[1] + delta_pos[1])
                new_cost = (current_cost + action.cost)
            
                if new_pos not in visited:
                    visited.add(new_pos)
                    queue.put((new_cost, new_pos))
                    branch[new_pos] = (new_cost, current_pos, action)
             
    path = []
    path_cost = 0
    if found:
        
        # retrace steps
        path = []
        n = goal
        path_cost = branch[n][0]
        while branch[n][1] != start:
            path.append(branch[n][2])
            n = branch[n][1]
        path.append(branch[n][2])
            
    return path[::-1], path_cost

### Executing the search

Run `uniform_cost()` and reference the grid to see if the path makes sense.

In [117]:
start = (0, 0)
goal = (4, 4)

grid = np.array([
    [0, 1, 0, 0, 0, 0],
    [0, 1, 0, 1, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 1, 0, 0, 1, 0],
    [0, 0, 0, 1, 0, 0],
])

In [118]:
path, path_cost = uniform_cost(grid, start, goal)
print(path_cost, path)

(0, (0, 0))
[<Action.DOWN: (1, 0, 1)>]
(1, (1, 0))
[<Action.UP: (-1, 0, 1)>, <Action.DOWN: (1, 0, 1)>]
(2, (2, 0))
[<Action.UP: (-1, 0, 1)>, <Action.DOWN: (1, 0, 1)>]
(3, (3, 0))
[<Action.UP: (-1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.DOWN_RIGHT: (1, 1, 1.4142135623730951)>]
(4, (4, 0))
[<Action.UP: (-1, 0, 1)>, <Action.RIGHT: (0, 1, 1)>]
(4.414213562373095, (4, 1))
[<Action.LEFT: (0, -1, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.UP_LEFT: (-1, -1, 1.4142135623730951)>, <Action.UP_RIGHT: (-1, 1, 1.4142135623730951)>]
(5.414213562373095, (4, 2))
[<Action.UP: (-1, 0, 1)>, <Action.LEFT: (0, -1, 1)>, <Action.UP_RIGHT: (-1, 1, 1.4142135623730951)>]
(5.82842712474619, (3, 2))
[<Action.UP: (-1, 0, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.UP_RIGHT: (-1, 1, 1.4142135623730951)>, <Action.DOWN_LEFT: (1, -1, 1.4142135623730951)>]
(6.82842712474619, (2, 2))
[<Action.UP: (-1, 0, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.DOWN_RIGHT: (1, 1, 1.414213562

In [119]:
# S -> start, G -> goal, O -> obstacle
visualize_path(grid, path, start)

array([['S', 'O', ' ', ' ', ' ', ' '],
       ['v', 'O', ' ', 'O', ' ', ' '],
       ['v', 'O', ' ', ' ', ' ', ' '],
       ['>', 'O', ' ', '>', 'O', ' '],
       [' ', '>', '>', 'O', 'G', ' ']], dtype='<U1')

[Solution](/notebooks/Cost-Solution.ipynb)